# Reading histograms

One of the most common tasks will be translating [ROOT](https://root.cern.ch) histograms into the HEPData format. `hepdata_lib` will help you with that, and this notebook will demonstrate how to do that.

As explained in the [Getting started notebook](Getting_started.ipynb), a `Submission` needs to exist or be created. Here, we'll just create one without any additional information:

In [21]:
from hepdata_lib import Submission
submission = Submission()

The plot will be a `Table`, in this example Figure 4a from page 12 (upper left) of the [publication](https://cms-results.web.cern.ch/cms-results/public-results/publications/B2G-17-009/index.html), which shows the distribution of the reconstructed B quark mass for the data as well as the individual simulated processes. Let's add all this, some more details as well as the actual plot (for thumbnail creation) to the `Table`:

In [22]:
# Submission text
submission.read_abstract("abstract.txt")
submission.add_link("Webpage with all figures and tables", "https://cms-results.web.cern.ch/cms-results/public-results/publications/SUS-21-002/")
#submission.add_link("arXiv", "Fill later")
#submission.add_record_id(1111, "inspire")

In [23]:
from hepdata_lib import Table
# table = Table("Figure 4a")
# table.description = "Distribution in the reconstructed B quark mass, after applying all selections to events with no forward jet, compared to the background distributions estimated before fitting. The plot refers to the low-mass mB analysis. The expectations for signal MC events are given by the blue histogram lines. Different contributions to background are indicated by the colour-filled histograms. The grey-hatched error band shows total uncertainties in the background expectation. The ratio of observations to background expectations is given in the lower panel, together with the total uncertainties prior to fitting, indicated by the grey-hatched band."
# table.location = "Data from Figure 4 (upper left), located on page 12."
# table.keywords["observables"] = ["N"]
# table.add_image("example_inputs/CMS-B2G-17-009_Figure_004-a.pdf")

table = Table("Figure 3a")
table.description = "bVeto MET distibution ..... to be filled"
table.location = "Data from Figure 3 (upper left), located on page 13."
table.keywords["observables"] = ["N"]
table.add_image("CMS-SUS-21-002_Figure_003-a.pdf")

The individual plot components are stored in different ROOT files, one for the individual background processes (one histogram per process plus the total), another one for the data, and the third for the signal process. All histograms here are of type [TH1](https://root.cern.ch/doc/master/classTH1.html), but you can also read in 2-dimensional [TH2](https://root.cern.ch/doc/master/classTH2.html) using `read_hist_2d(...)` instead of `read_hist_1d(...)`:

In [24]:
from hepdata_lib import RootFileReader

# reader = RootFileReader("example_inputs/mlfit_lm_1000.root")
# reader_data = RootFileReader("example_inputs/Data_cat0_singleH.root")
# reader_signal = RootFileReader("example_inputs/BprimeBToHB1000_cat0_singleH.root")

# TotalBackground = reader.read_hist_1d("shapes_prefit/cat0_singleH/total_background")
# TT = reader.read_hist_1d("shapes_prefit/cat0_singleH/TT")
# QCD = reader.read_hist_1d("shapes_prefit/cat0_singleH/QCDTT")
# WJets = reader.read_hist_1d("shapes_prefit/cat0_singleH/WJets")
# ZJets = reader.read_hist_1d("shapes_prefit/cat0_singleH/ZJets")

# Data = reader_data.read_hist_1d("h_bprimemass_SRlm")

# signal = reader_signal.read_hist_1d("h_bprimemass_SRlm")

reader = RootFileReader("combined_PostFitPredictions_Run2_Mask.root")
reader_signalWZ = RootFileReader("TChiWZ_600_1_MCRun2.root")
reader_signalWW = RootFileReader("TChiWW_500_1_MCRun2.root")

TotalBackground = reader.read_hist_1d("TotalBG")
ZEROONERES = reader.read_hist_1d("zeroOneres")
TWORES = reader.read_hist_1d("Twores")

Data = reader.read_hist_1d("DataFit")

signalWZ = reader_signalWZ.read_hist_1d("METvBin")
signalWW = reader_signalWW.read_hist_1d("METvBin")

The content of the histograms is stored as a dictionary, with keys `x` (bin center), `y` (bin value or for `TH2` the bin center of the 2nd dimension), `z` (`TH2` only: bin value), as well as the bin errors `dy` (`dz` for `TH2`). Furthermore, the lower and upper bin edges (`x_edges`, for `TH2` also `y_edges`) are stored for each bin:

In [25]:
TotalBackground.keys()

dict_keys(['x', 'y', 'x_edges', 'x_labels', 'dy'])

The `RootFileReader` automatically recognises if the histogram has symmetric or assymmetric errors based on [TH1::GetBinErrorOption()](https://root.cern.ch/doc/master/classTH1.html#ac6e38c12259ab72c0d574614ee5a61c7). Symmetric errors are returned if this returns `TH1::kNormal`, in this case (as for the example here) the errors are a plain `float` per bin, otherwise a `tuple` of `float`. The bin edges are always stored as `tuple`:

In [26]:
from __future__ import print_function
for key in TotalBackground.keys():
    print(key, type(TotalBackground[key]), type(TotalBackground[key][0]))

x <class 'list'> <class 'float'>
y <class 'list'> <class 'float'>
x_edges <class 'list'> <class 'tuple'>
x_labels <class 'list'> <class 'str'>
dy <class 'list'> <class 'float'>


Now define the variables:

In [27]:
from hepdata_lib import Variable, Uncertainty

#-----------

# x-axis: MET
#print(Data["x_edges"])
xBins = Variable("$p_{miss}^{T}$", is_independent=True, is_binned=True, units="GeV")
# temp = []
# for i in range(0,len(Data["x"])-1):
#     temp.append((Data["x"][i],Data["x"][i+1]))
# xBins.values = temp
xBins.values = Data["x_edges"]

sigWZ = Variable("Number of WZ signal events", is_independent=False, is_binned=False, units="")
sigWZ.values = signalWZ["y"]

# y-axis: N events
sigWW = Variable("Number of WW signal events", is_independent=False, is_binned=False, units="")
sigWW.values = signalWW["y"]

totalbackground = Variable("Number of background events", is_independent=False, is_binned=False, units="")
totalbackground.values = TotalBackground["y"]

zeroOneRes = Variable("Number of 0,1 res BG events", is_independent=False, is_binned=False, units="")
zeroOneRes.values = ZEROONERES["y"]

twoRes = Variable("Number of 2 res BG events", is_independent=False, is_binned=False, units="")
twoRes.values = TWORES["y"]

data = Variable("Number of data events", is_independent=False, is_binned=False, units="")
data.values = Data["y"]

For the data as well as the background total, we will also provide the associated uncertainties:

In [28]:
from hepdata_lib import Uncertainty

unc_totalbackground = Uncertainty("Total uncertainty", is_symmetric=True)
unc_totalbackground.values = TotalBackground["dy"]

unc_data = Uncertainty("Data uncertainty", is_symmetric=True)
unc_data.values = Data["dy"]

totalbackground.add_uncertainty(unc_totalbackground)
data.add_uncertainty(unc_data)

Now we can add the variables to the `Table` and the `Table` to the `Submission`, and create the files. Please refer to the [Getting started notebook](Getting_started.ipynb) for a complete example.

In [29]:
#table.add_variable(mmed)
table.add_variable(xBins)
table.add_variable(zeroOneRes)
table.add_variable(twoRes)
table.add_variable(totalbackground)
table.add_variable(sigWZ)
table.add_variable(sigWW)
table.add_variable(data)

submission.add_table(table)

#submission.create_files("submission",remove_old=True)

In [30]:
!head submission/figure_3a.yaml

head: cannot open 'submission/figure_3a.yaml' for reading: No such file or directory


In [31]:
def write_obs_contours(inputfile, tablename, desc, loc, kw, addImage):
    table = Table(tablename)
    table.description = desc
    table.location = loc
    table.keywords["observables"] = kw
    table.add_image(addImage)
    reader = RootFileReader(inputfile)
    if "TChiWH" in inputfile:
        OBSWW = reader.read_graph("graph_smoothed_Obs_1")
        OBSPWW = reader.read_graph("graph_smoothed_ObsP_1")
        OBSMWW = reader.read_graph("graph_smoothed_ObsM_1")
    else:
        OBSWW = reader.read_graph("graph_smoothed_Obs")
        OBSPWW = reader.read_graph("graph_smoothed_ObsP")
        OBSMWW = reader.read_graph("graph_smoothed_ObsM")
    
    # x-axis: Mass of NLSP
    mnlsp = Variable("m_NSLP", is_independent=True, is_binned=False, units="GeV")
    mnlsp.values = OBSWW["x"] + OBSPWW["x"] + OBSMWW["x"]
    
    #y-axis: obs limit contours
    obsWW = Variable("Obs. limit", is_independent=False, is_binned=False, units="")
    obsWW.values = OBSWW["y"] + ['~'] * len(OBSPWW["x"] + OBSMWW["x"])

    obsPWW = Variable("Obs. +1 sigma limit", is_independent=False, is_binned=False, units="")
    obsPWW.values = ['~'] * len(OBSWW["x"]) + OBSPWW["y"] + ['~'] * len(OBSMWW["x"])

    obsMWW = Variable("Obs. -1 sigma limit", is_independent=False, is_binned=False, units="")
    obsMWW.values = ['~'] * len(OBSWW["x"] + OBSPWW["x"]) + OBSMWW["y"] 
    
    table.add_variable(mnlsp)
    table.add_variable(obsWW)
    table.add_variable(obsPWW)
    table.add_variable(obsMWW)

    submission.add_table(table)

In [32]:
def write_exp_contours(inputfile, tablename, desc, loc, kw, addImage):
    table = Table(tablename)
    table.description = desc
    table.location = loc
    table.keywords["observables"] = kw
    table.add_image(addImage)
    reader = RootFileReader(inputfile)


    EXPWW = reader.read_graph("graph_smoothed_Exp")
    EXPPWW = reader.read_graph("graph_smoothed_ExpP")
    EXPMWW = reader.read_graph("graph_smoothed_ExpM")
    
    # x-axis: Mass of NLSP
    mnlsp = Variable("m_NLSP", is_independent=True, is_binned=False, units="GeV")
    mnlsp.values = EXPWW["x"] + EXPPWW["x"] + EXPMWW["x"]

    #y-axis: exp limit contours
    expWW = Variable("Obs. limit", is_independent=False, is_binned=False, units="")
    expWW.values = EXPWW["y"] + ['~'] * len(EXPPWW["x"] + EXPMWW["x"])

    expPWW = Variable("Obs. +1 sigma limit", is_independent=False, is_binned=False, units="")
    expPWW.values = ['~'] * len(EXPWW["x"]) + EXPPWW["y"] + ['~'] * len(EXPMWW["x"])

    expMWW = Variable("Obs. -1 sigma limit", is_independent=False, is_binned=False, units="")
    expMWW.values = ['~'] * len(EXPWW["x"] + EXPPWW["x"]) + EXPMWW["y"] 

    table.add_variable(mnlsp)
    table.add_variable(expWW)
    table.add_variable(expPWW)
    table.add_variable(expMWW)


    submission.add_table(table)

In [33]:
def obs_heatmap(inputfile, tablename, desc, loc, kw, addImage):
    table = Table(tablename)
    table.description = desc
    table.location = loc
    table.keywords["observables"] = kw
    table.add_image(addImage)
    reader = RootFileReader(inputfile)

    OBSXSEC = reader.read_hist_2d("hXsec_obs_corr")
    #print(OBSXSEC["x_edges"])
    mnlsp = Variable("m_NLSP", is_independent=True, is_binned=True, units="GeV")
    mnlsp.values = OBSXSEC["x_edges"]
    
    mlsp = Variable("m_LSP", is_independent=True, is_binned=True, units="GeV")
    mlsp.values = OBSXSEC["y_edges"]
    
    obsxsec = Variable("XSec", is_independent=False, is_binned=False, units="")
    obsxsec.values = OBSXSEC["z"]
    
    table.add_variable(mnlsp)
    table.add_variable(mlsp)
    table.add_variable(obsxsec)
    
    submission.add_table(table)

In [34]:
#TChiWW
write_obs_contours("afterCWR_TChiWW.root","Figure 4a Observed Lines","TChiWW obs limit plot ..... to be filled",\
                   "Data from Figure 4 (upper left), located on page 14.", ["Cross-section"], "Figure_004-a.pdf")

write_exp_contours("afterCWR_TChiWW.root","Figure 4a Expected Lines","TChiWW exp limit plot ..... to be filled",\
                   "Data from Figure 4 (upper left), located on page 14.", ["Cross-section"], "Figure_004-a.pdf")

obs_heatmap("afterCWR_TChiWW.root","Figure 4a Observed","TChiWW obs 95% CL on cross section ..... to be filled",\
                   "Data from Figure 4 (upper left), located on page 14.", ["Cross-section"], "Figure_004-a.pdf")

In [35]:
#TChiWZ
write_obs_contours("afterCWR_TChiWZ.root","Figure 4b Observed Lines","TChiWZ obs limit plot ..... to be filled",\
                   "Data from Figure 4 (upper right), located on page 14.", ["Cross-section"], "Figure_004-b.pdf")

write_exp_contours("afterCWR_TChiWZ.root","Figure 4b Expected Lines","TChiWZ exp limit plot ..... to be filled",\
                   "Data from Figure 4 (upper  right), located on page 14.", ["Cross-section"], "Figure_004-b.pdf")

obs_heatmap("afterCWR_TChiWZ.root","Figure 4b Observed","TChiWZ obs limit plot ..... to be filled",\
                   "Data from Figure 4 (upper right), located on page 14.", ["Cross-section"], "Figure_004-b.pdf")

In [36]:
# #TChiWH
write_obs_contours("afterCWR_TChiWH.root","Figure 4c Observed Lines","TChiWH obs limit plot ..... to be filled",\
                   "Data from Figure 4 (lower), located on page 14.", ["Cross-section"], "Figure_004-c.pdf")

write_exp_contours("afterCWR_TChiWH.root","Figure 4c Expected Lines","TChiWH exp limit plot ..... to be filled",\
                   "Data from Figure 4 (lower), located on page 14.", ["Cross-section"], "Figure_004-c.pdf")

obs_heatmap("afterCWR_TChiWH.root","Figure 4c Observed","TChiWH obs limit plot ..... to be filled",\
                   "Data from Figure 4 (lower), located on page 14.", ["Cross-section"], "Figure_004-c.pdf")

In [37]:
# TChiWW + WZ Wino combined Fig 5 left
write_obs_contours("test_TChiWW_WZ_afterCWR.root","Figure 5a WW WZ Observed Lines","TChiWW + TChiWZ obs limit plot ..... to be filled",\
                   "Data from Figure 5 (left), located on page 14.", ["Cross-section"], "Figure_005-a.pdf")

write_exp_contours("test_TChiWW_WZ_afterCWR.root","Figure 5a WW WZ Expected Lines","TChiWW + TChiWZ exp limit plot ..... to be filled",\
                   "Data from Figure 5 (left), located on page 14.", ["Cross-section"], "Figure_005-a.pdf")

obs_heatmap("test_TChiWW_WZ_afterCWR.root","Figure 5a WW WZ Observed","TChiWW + TChiWZ obs limit plot ..... to be filled",\
                   "Data from Figure 5 (left), located on page 14.", ["Cross-section"], "Figure_005-a.pdf")

In [38]:
# TChiWW + WH Wino combined Fig 5 left
write_obs_contours("test_TChiWW_WH_afterCWR.root","Figure 5a WW WH Observed Lines","TChiWW + TChiWH obs limit plot ..... to be filled",\
                   "Data from Figure 5 (left), located on page 15.", ["Cross-section"], "Figure_005-a.pdf")

write_exp_contours("test_TChiWW_WH_afterCWR.root","Figure 5a WW WH Expected Lines","TChiWW + TChiWH exp limit plot ..... to be filled",\
                   "Data from Figure 5 (left), located on page 15.", ["Cross-section"], "Figure_005-a.pdf")

obs_heatmap("test_TChiWW_WH_afterCWR.root","Figure 5a WW WH Observed","TChiWW + TChiWH obs limit plot ..... to be filled",\
                   "Data from Figure 5 (left), located on page 15.", ["Cross-section"], "Figure_005-a.pdf")

In [39]:
# Hino combined Fig 5 right
write_obs_contours("test_TChiWW_WZ_WH_HZ_afterCWR.root","Figure 5b Observed Lines","Higgsino obs limit plot ..... to be filled",\
                   "Data from Figure 5 (right), located on page 15.", ["Cross-section"], "Figure_005-b.pdf")

write_exp_contours("test_TChiWW_WZ_WH_HZ_afterCWR.root","Figure 5b Expected Lines","Higgsino exp limit plot ..... to be filled",\
                   "Data from Figure 5 (right), located on page 15.", ["Cross-section"], "Figure_005-b.pdf")

obs_heatmap("test_TChiWW_WZ_WH_HZ_afterCWR.root","Figure 5b Observed","Higgsino obs limit plot ..... to be filled",\
                   "Data from Figure 5 (right), located on page 15.", ["Cross-section"], "Figure_005-b.pdf")

In [40]:
submission.create_files("Test2",remove_old=True)

Full-size PNG file Test2/Figure_004-a.png is newer than its source file.                        Remove the thumbnail file or use create_files(remove_old=True)                           to force recreation.
Thumbnail PNG file Test2/thumb_Figure_004-a.png is newer than its source file.                        Remove the thumbnail file or use create_files(remove_old=True)                           to force recreation.
Full-size PNG file Test2/Figure_004-a.png is newer than its source file.                        Remove the thumbnail file or use create_files(remove_old=True)                           to force recreation.
Thumbnail PNG file Test2/thumb_Figure_004-a.png is newer than its source file.                        Remove the thumbnail file or use create_files(remove_old=True)                           to force recreation.
Full-size PNG file Test2/Figure_004-b.png is newer than its source file.                        Remove the thumbnail file or use create_files(remove_old=True)      